In [36]:
from loading import load_directory
from kmers import encode2kmr, encode2str, encode2scr
from main import similarity, jaccard, method1, method2
from collections import Counter
import time
import numpy as np
import pandas as pd

# Load all the files in a dictionary
files = load_directory("data")
k = 21
names = ['ASM584', 'ASM694', 'ASM886', 'ASM2216', 'ASM824478']
matrix_jaccard = np.zeros((len(files), len(files)))
matrix_similarity1 = np.zeros((len(files), len(files)))
matrix_similarity2 = np.zeros((len(files), len(files)))
filenames = list(files.keys())
for i in range(len(files)):
    for j in range(i+1, len(files)):
        start_t = time.time()
        #A, inter, B = method1(
        #    files[filenames[i]], files[filenames[j]], k)
        A, inter, B = method2(files[filenames[i]], files[filenames[j]], k)
        end_t = time.time()
        print("Names of .fna files for comparison: " +
              filenames[i] + ' ' + filenames[j])
        matrix_jaccard[i][j] = jaccard(A, inter, B)
        print("Jaccard score: " + str(jaccard(A, inter, B)))
        matrix_similarity1[i][j] = similarity(A, inter, B)[0]
        matrix_similarity2[i][j] = similarity(A, inter, B)[1]
        print("Similarity score: " + str(similarity(A, inter, B)))
        print("Time spent: " + str(end_t-start_t))

df_jaccard = pd.DataFrame(
    matrix_jaccard, index=names, columns=names)
df_similarity1 = pd.DataFrame(
    matrix_similarity1, index=names, columns=names)
df_similarity2 = pd.DataFrame(
    matrix_similarity2, index=names, columns=names)

Names of .fna files for comparison: GCF_000006945.2_ASM694v2_genomic.fna GCF_000005845.2_ASM584v2_genomic.fna
Jaccard score: 0.01910907532349267
Similarity score: (0.036328582654917445, 0.0387527490330987)
Time spent: 12.849003791809082
Names of .fna files for comparison: GCF_000006945.2_ASM694v2_genomic.fna GCF_008244785.1_ASM824478v1_genomic.fna
Jaccard score: 0.9377857123269104
Similarity score: (0.9600075777114302, 0.9759113420442428)
Time spent: 12.518712282180786
Names of .fna files for comparison: GCF_000006945.2_ASM694v2_genomic.fna GCF_000022165.1_ASM2216v1_genomic.fna
Jaccard score: 0.9570811316736416
Similarity score: (0.979325692743594, 0.9768174425980902)
Time spent: 12.378711223602295
Names of .fna files for comparison: GCF_000006945.2_ASM694v2_genomic.fna GCF_000008865.2_ASM886v2_genomic.fna
Jaccard score: 0.01714739962081993
Similarity score: (0.035906678625663276, 0.03177840715620551)
Time spent: 15.201646089553833
Names of .fna files for comparison: GCF_000005845.2_AS

In [37]:
last_df_jaccard = df_jaccard + df_jaccard.T + np.eye(5)
last_df_jaccard

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000000,0.019109,0.937786,0.957081,0.017147
ASM694,0.019109,1.000000,0.019342,0.019153,0.436482
ASM886,0.937786,0.019342,1.000000,0.979695,0.017273
ASM2216,0.957081,0.019153,0.979695,1.000000,0.017154
ASM824478,0.017147,0.436482,0.017273,0.017154,1.000000


In [38]:
last_df_similarity1 = df_similarity1 + df_similarity1.T + np.eye(5)
last_df_similarity1

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000000,0.036329,0.960008,0.979326,0.035907
ASM694,0.036329,1.000000,0.038886,0.038891,0.670093
ASM886,0.960008,0.038886,1.000000,0.999233,0.036483
ASM2216,0.979326,0.038891,0.999233,1.000000,0.035872
ASM824478,0.035907,0.670093,0.036483,0.035872,1.000000


In [39]:
last_df_similarity2 = df_similarity2 + df_similarity2.T + np.eye(5)
last_df_similarity2

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000000,0.038753,0.975911,0.976817,0.031778
ASM694,0.038753,1.000000,0.037058,0.036365,0.555953
ASM886,0.975911,0.037058,1.000000,0.980432,0.031762
ASM2216,0.976817,0.036365,0.980432,1.000000,0.031829
ASM824478,0.031778,0.555953,0.031762,0.031829,1.000000


In [1]:
from loading import load_directory
from kmers import encode2kmr, encode2str, encode2scr
from main import similarity, jaccard, sketch2, sketch3, comparesortedlists
from collections import Counter
import time
import numpy as np
import pandas as pd
import heapq

# Load all the files in a dictionary
files = load_directory("data")
k = 21
names = ['ASM584', 'ASM694', 'ASM886', 'ASM2216', 'ASM824478']
matrix_sketch2_jaccard = np.zeros((len(files), len(files)))
matrix_sketch2_similarity1 = np.zeros((len(files), len(files)))
matrix_sketch2_similarity2 = np.zeros((len(files), len(files)))
matrix_sketch3_jaccard = np.zeros((len(files), len(files)))
matrix_sketch3_similarity1 = np.zeros((len(files), len(files)))
matrix_sketch3_similarity2 = np.zeros((len(files), len(files)))
filenames = list(files.keys())
s = 1000

for i in range(len(files)):
    for j in range(i+1, len(files)):
        start_t = time.time()
        #A = sketch1(s, files[filenames[i]], k)
        #B = sketch1(s, files[filenames[j]], k)
        #inter = comparesortedlists(A, B)
        A = sketch2(s, files[filenames[i]], k)
        B = sketch2(s, files[filenames[j]], k)
        inter = comparesortedlists(A, B)
        matrix_sketch2_jaccard[i][j] = jaccard(A, inter, B)
        matrix_sketch2_similarity1[i][j] = similarity(A, inter, B)[0]
        matrix_sketch2_similarity2[i][j] = similarity(A, inter, B)[1]

        print("Jaccard score: " + str(matrix_sketch2_jaccard[i][j]))
        print("Similarity score: " +
              str(matrix_sketch2_similarity1[i][j]) + str(matrix_sketch2_similarity2[i][j]))

        A = sketch3(s, files[filenames[i]], k)
        B = sketch3(s, files[filenames[j]], k)
        inter = comparesortedlists(A, B)
        matrix_sketch3_jaccard[i][j] = jaccard(A, inter, B)
        matrix_sketch3_similarity1[i][j] = similarity(A, inter, B)[0]
        matrix_sketch3_similarity2[i][j] = similarity(A, inter, B)[1]
        end_t = time.time()
        print("Jaccard score: " + str(matrix_sketch3_jaccard[i][j]))
        print("Similarity score: " +
              str(matrix_sketch3_similarity1[i][j]) + str(matrix_sketch3_similarity2[i][j]))
        print("Time spent: " + str(end_t-start_t))

df_sketch2_jaccard = pd.DataFrame(
    matrix_sketch2_jaccard, index=names, columns=names)
df_sketch2_similarity1 = pd.DataFrame(
    matrix_sketch2_similarity1, index=names, columns=names)
df_sketch2_similarity2 = pd.DataFrame(
    matrix_sketch2_similarity2, index=names, columns=names)
df_sketch3_jaccard = pd.DataFrame(
    matrix_sketch3_jaccard, index=names, columns=names)
df_sketch3_similarity1 = pd.DataFrame(
    matrix_sketch3_similarity1, index=names, columns=names)
df_sketch3_similarity2 = pd.DataFrame(
    matrix_sketch3_similarity2, index=names, columns=names)

Jaccard score: 0.01574403250380904
Similarity score: 0.0310.031
Jaccard score: 0.008572869389813415
Similarity score: 0.0170.017
Time spent: 18.539507627487183
Jaccard score: 0.9083969465648855
Similarity score: 0.9520.952
Jaccard score: 0.8535681186283596
Similarity score: 0.9210.921
Time spent: 18.53535008430481
Jaccard score: 0.9120458891013384
Similarity score: 0.9540.954
Jaccard score: 0.8832391713747646
Similarity score: 0.9380.938
Time spent: 17.445492029190063
Jaccard score: 0.013684744044602128
Similarity score: 0.0270.027
Jaccard score: 0.008064516129032258
Similarity score: 0.0160.016
Time spent: 19.337865829467773
Jaccard score: 0.01574403250380904
Similarity score: 0.0310.031
Jaccard score: 0.008064516129032258
Similarity score: 0.0160.016
Time spent: 16.99017596244812
Jaccard score: 0.01574403250380904
Similarity score: 0.0310.031
Jaccard score: 0.008064516129032258
Similarity score: 0.0160.016
Time spent: 18.483818531036377
Jaccard score: 0.24610591900311526
Similarity s

In [2]:
last_df_sketch2_jaccard = df_sketch2_jaccard + df_sketch2_jaccard.T + np.eye(5)
last_df_sketch2_jaccard

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000000,0.015744,0.908397,0.912046,0.013685
ASM694,0.015744,1.000000,0.015744,0.015744,0.246106
ASM886,0.908397,0.015744,1.000000,0.962709,0.013685
ASM2216,0.912046,0.015744,0.962709,1.000000,0.013685
ASM824478,0.013685,0.246106,0.013685,0.013685,1.000000


In [3]:
last_df_sketch2_similarity1 = df_sketch2_similarity1 + df_sketch2_similarity1.T + np.eye(5)
last_df_sketch2_similarity1

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000,0.031,0.952,0.954,0.027
ASM694,0.031,1.000,0.031,0.031,0.395
ASM886,0.952,0.031,1.000,0.981,0.027
ASM2216,0.954,0.031,0.981,1.000,0.027
ASM824478,0.027,0.395,0.027,0.027,1.000


In [4]:
last_df_sketch2_similarity2 = df_sketch2_similarity2 + df_sketch2_similarity2.T + np.eye(5)
last_df_sketch2_similarity2

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000,0.031,0.952,0.954,0.027
ASM694,0.031,1.000,0.031,0.031,0.395
ASM886,0.952,0.031,1.000,0.981,0.027
ASM2216,0.954,0.031,0.981,1.000,0.027
ASM824478,0.027,0.395,0.027,0.027,1.000


In [5]:
last_df_sketch3_jaccard = df_sketch3_jaccard + df_sketch3_jaccard.T + np.eye(5)
last_df_sketch3_jaccard

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000000,0.008573,0.853568,0.883239,0.008065
ASM694,0.008573,1.000000,0.008065,0.008065,0.182732
ASM886,0.853568,0.008065,1.000000,0.953125,0.008065
ASM2216,0.883239,0.008065,0.953125,1.000000,0.008065
ASM824478,0.008065,0.182732,0.008065,0.008065,1.000000


In [6]:
last_df_sketch3_similarity1 = df_sketch3_similarity1 + df_sketch3_similarity1.T + np.eye(5)
last_df_sketch3_similarity1

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000,0.017,0.921,0.938,0.016
ASM694,0.017,1.000,0.016,0.016,0.309
ASM886,0.921,0.016,1.000,0.976,0.016
ASM2216,0.938,0.016,0.976,1.000,0.016
ASM824478,0.016,0.309,0.016,0.016,1.000


In [7]:
last_df_sketch3_similarity2 = df_sketch3_similarity2 + df_sketch3_similarity2.T + np.eye(5)
last_df_sketch3_similarity2

,ASM584,ASM694,ASM886,ASM2216,ASM824478
ASM584,1.000,0.017,0.921,0.938,0.016
ASM694,0.017,1.000,0.016,0.016,0.309
ASM886,0.921,0.016,1.000,0.976,0.016
ASM2216,0.938,0.016,0.976,1.000,0.016
ASM824478,0.016,0.309,0.016,0.016,1.000
